In [ ]:
import Pkg
Pkg.activate("..\\..\\juMLia")
import MLDatasets: FashionMNIST, convert2image
using Plots, DataFrames, ColorSchemes, ImageShow

Neural networks are overkill for datasets on the scale of Wine. But they're useful in things like computer vision here, where we're classifying tiny images of different fashion items.

In [ ]:
traindata = FashionMNIST(:train)
testdata = FashionMNIST(:test)

In [ ]:
convert2image(traindata, 1)

We're going to use the sigmoid function as the activation function for all the neurons in the network, so we need a way to get a value between 0 and 9 out of a bunch of binary classifiers. We do it by encoding the value as a bit sequence, where the index of the only on bit corresponds to the value. When we get an output from the network, we'll call argmax on the float values we get from it to get the value.

In [ ]:
function onehotencodedigits(digit::Int)
    digitencoded = BitArray(zeros(10))
    digitencoded[digit+1] = 1
    return digitencoded
end

"One-hot encodes a vector of integers 0-9."
function onehotencodedigits(digitlist::Vector{<:Int})
    return onehotencodedigits.(digitlist)
end

function onehotdecodedigits(digitsencoded::BitArray)
    return argmax(digitsencoded) - 1
end

function onehotdecodedigits(digitsencoded::Vector{<:BitArray})
    return onehotdecodedigits.(digitsencoded)
end

function onehotdecodedigits(digitsencoded::Vector{<:Number})
    return argmax(digitsencoded) - 1
end

One simplification we make is we flatten the 2D image into one-dimensional vectors.

In [ ]:
trainfeatures = vec.(traindata.features[:,:,ftridx] for ftridx in 1:size(traindata.features, 3))
trainlabels = onehotencodedigits.(traindata.targets)

testfeatures = vec.(testdata.features[:,:,ftridx] for ftridx in 1:size(testdata.features, 3))
testlabels = onehotencodedigits.(testdata.targets)

We descend down the loss function according to the gradient defined by the first derivative of the sigmoid function (which happens to be nice and closed-form).

In [ ]:
sigmoid(value) = 1.0 ./ (1.0 .+ exp.(-value))

sigmoid_firstderiv(value) = sigmoid(value) .* (1.0 .- sigmoid(value))

"""Mean squared error between `predictions` and `targets`. Used as a 
loss function."""
meansquarederror(predictions, targets) = 0.5 * sum((predictions .- targets).^2)

We implement a dense, deep neural network, where every neuron at a particular layer is connected to every neuron at the following layer and there are at least two hidden layers (hidden meaning between the input and output layers). Generally, each layer might have its own activation functions and gradient functions. Because of the density of the network, we store the weights as matrices--going from one layer to the next corresponds to multiplying the output vector of one layer by the weight matrix and adding the bias vector, then applying the activation function to that.

In [ ]:
abstract type DenseNeuralNetwork end

# Struct of arrays
mutable struct DenseNeuralNetworkSoA{LossF} <: DenseNeuralNetwork
    const activationfunctions::Vector{Function}
    const gradients::Vector{Function}
    const loss::LossF
    weights::Vector{Matrix{Float64}} # Can't store as array because matrices may have different sizes
    bias::Vector{Vector{Float64}}
    previousweights::Vector{Matrix{Float64}}
    previousbias::Vector{Vector{Float64}}
    losshistory::Vector{Float64}
    prevlosshistory::Vector{Float64}
end 

In [ ]:
function initweights(layerlengths; randweights = true)
    numweightmatrices = length(layerlengths) - 1
    weights = Vector{Matrix{Float64}}(undef, numweightmatrices)
    bias = Vector{Vector{Float64}}(undef, numweightmatrices)
    for layerindex in 1:numweightmatrices
        if randweights
            weights[layerindex] = (sqrt(2 / layerlengths[layerindex]) 
                                    .* randn((layerlengths[layerindex+1], 
                                                layerlengths[layerindex])))
            
            bias[layerindex] = (sqrt(2 / layerlengths[layerindex])
                                .* randn(layerlengths[layerindex+1]))
        else
            weights[layerindex] = zeros(layerlengths[layerindex+1], 
                                        layerlengths[layerindex])
            bias[layerindex] = zeros(layerlengths[layerindex+1])
        end
    end
    return (weights, bias)
end

In [ ]:
function DenseNeuralNetworkSoA(activationfunctions::Vector,
    gradientfunctions::Vector,
    lossfunction,
    weights::Vector{Matrix{Float64}},
    bias::Vector{Vector{Float64}})
    if length(activationfunctions) != length(gradientfunctions)
        error("Function lists must be of the same length. \
        Given lengths are $(length(activationfunctions)) \
        and $(length(gradientfunctions)).")
    end

    if length(weights) != length(bias)
        error("Weights and bias vectors must be of compatible length. \
        Given lengths are $(length(weights)) \
        and $(length(bias)).")
    end

    if length(weights) != length(activationfunctions)
        error("Number of weight matrices and number of functions \
        be the same. Given are $(length(weights)) \
        weights and $(length(activationfunctions)) \
        functions.")
    end

    return DenseNeuralNetworkSoA{typeof(lossfunction)}(activationfunctions,
        gradientfunctions,
        lossfunction,
        weights, bias, weights, bias,
        [], [])
end

function DenseNeuralNetworkSoA(activationfunctions::Vector,
    gradientfunctions::Vector,
    lossfunction,
    numnodesperlayer)

    numlayers = length(activationfunctions) + 1

    if (length(numnodesperlayer) != numlayers
        &&
        length(numnodesperlayer) > 1)
        error("Must be either a constant node count or as many 
        node counts as there are layers. \
        Given layer count is $(numlayers) \
        and the node counts specified are of length $(length(numnodesperlayer)).")
    end

    weights = nothing
    bias = nothing
    if isa(numnodesperlayer, <:Int)
        (weights, bias) = initweights(fill(numnodesperlayer, numlayers))
    else
        (weights, bias) = initweights(numnodesperlayer)
    end

    return DenseNeuralNetworkSoA(activationfunctions, gradientfunctions,
        lossfunction,
        weights, bias, weights, bias,
        [], [])
end

function DenseNeuralNetworkSoA(activationfunction,
    gradientfunction,
    lossfunction,
    weights::Vector{Matrix{Float64}},
    bias::Vector{Vector{Float64}})

    numweightmatrices = length(weights)
    return DenseNeuralNetworkSoA(fill(activationfunction, numweightmatrices),
        fill(gradientfunction, numweightmatrices),
        lossfunction, weights, bias)
end

function DenseNeuralNetworkSoA(activationfunction,
    gradientfunction,
    lossfunction,
    numnodesperlayer::Vector{<:Int})

    numlayers = length(numnodesperlayer)
    return DenseNeuralNetworkSoA(fill(activationfunction, numlayers - 1),
        fill(gradientfunction, numlayers - 1),
        lossfunction, weights, bias)
end

function DenseNeuralNetworkSoA(weights::Vector{Matrix{Float64}},
    bias::Vector{Vector{Float64}})
    return DenseNeuralNetworkSoA(sigmoid, sigmoid_firstderiv, meansquarederror,
        weights, bias)
end

function DenseNeuralNetworkSoA(inputlength::Int,
    outputlength::Int;
    numhiddenlayers::Int=2,
    hiddenlayerlengths::Vector{<:Int}=
    fill(round(Int, √(inputlength * outputlength)),
        numhiddenlayers)
)
    numweightmatrices = 1 + numhiddenlayers
    if length(hiddenlayerlengths) != numhiddenlayers
        error("`hiddenlayerlengths` vector must have $(numhiddenlayers) \
        entries.")
    end
    layerlengths = [inputlength; hiddenlayerlengths; outputlength]
    return DenseNeuralNetworkSoA(initweights(layerlengths)...)
end

function DenseNeuralNetworkSoA(nodecounts::Vector{<:Int})
    return DenseNeuralNetworkSoA(initweights(nodecounts)...)
end

In [ ]:
function numlayers(neurnet::DenseNeuralNetworkSoA)
    return length(neurnet.weights) + 1
end

function weightsizes(neurnet::DenseNeuralNetworkSoA)
    return [size(weightmat) for weightmat in neurnet.weights]
end

function numnodesinlayer(neurnet::DenseNeuralNetworkSoA, layerindex::Int)
    if layerindex <= length(neurnet.weights)
        return size(neurnet.weights[layerindex])[end]
    elseif layerindex == length(neurnet.weights) + 1
        return size(neurnet.weights[layerindex])[begin]
    else
        return NaN
    end
end

function numnodesalllayers(neurnet::DenseNeuralNetworkSoA)
    return [size(neurnet.weights[begin])[end]; 
            [size(weightmat)[begin] for weightmat in neurnet.weights]]
end


A forward pass constitutes a single run through the network: you give it a single feature vector, the network multiplies through its weight matrices and passes the results through its activation functions, then the network outputs a label vector.

In [ ]:
function forwardpass(neurnet::DenseNeuralNetworkSoA, input)
    output = input
    for (weightmat, biasvec, activate) in zip(neurnet.weights, 
                                              neurnet.bias, 
                                              neurnet.activationfunctions)
        # Creating all these intermediate vectors of potentially very large size...
        # ...is fine! Happens very quickly and is actually a little faster than 
        # copying into an already-existing array.
        output = activate(weightmat * output .+ biasvec)
    end
    return output
end

function forwardpass_keepvals!(neurnet::DenseNeuralNetworkSoA, input, 
                                preactivationvecs::Array{T}, 
                                postactivationvecs) where T <: Array{U} where U
    output = postactivationvecs[1] = input
    preactivationvecs[1] = [zero(eltype(eltype(preactivationvecs)))]

    for (i, (weightmat, biasvec, activate)) in enumerate(zip(neurnet.weights, 
                                                                neurnet.bias, 
                                                                neurnet.activationfunctions))
        preactivationvecs[i+1] = weightmat * output .+ biasvec
        output = postactivationvecs[i+1] = activate(preactivationvecs[i+1])
    end

    return (output, preactivationvecs, postactivationvecs)
end

function forwardpass_keepvals(neurnet, input)
    preactivationvecs = Vector{Vector{Float64}}(undef, numlayers(neurnet))
    postactivationvecs = Vector{Vector{Float64}}(undef, numlayers(neurnet))

    return forwardpass_keepvals!(neurnet, input, preactivationvecs, postactivationvecs)
end

The network predicting a value, then, only requires performing a forward pass.

In [ ]:
function predictsingle(neurnet, input)
    return forwardpass(neurnet, input)
end

function predictmultiple(neurnet, inputs::Vector)
    return [forwardpass(neurnet, input) for input in inputs]
end

function predictmultiple(neurnet, inputs::Matrix)
    return [forwardpass(neurnet, view(inputs, :, colindex)) for colindex in 1:size(inputs, 2)]
end

function predictsingle_keepvals(neurnet, input)
    return forwardpass_keepvals(neurnet, input)
end

function predictmultiple_keepvals(neurnet, inputs::Vector)
    return [forwardpass_keepvals(neurnet, view(inputs, :, colindex)) for colindex in 1:size(inputs, 2)]
end

function predict(neurnet, input; keepvals=false)
    if !keepvals
        return predictsingle(neurnet, input)
    else
        return predictsingle_keepvals(neurnet, input)
    end
end

# Generally if you give it a vector it assumes you want multiple predictions...
function predict(neurnet, inputs::Vector; keepvals=false)
    if !keepvals
        return predictmultiple(neurnet, inputs)
    else
        return predictmultiple_keepvals(neurnet, inputs)
    end
end

# ...but if it's a vector of numbers, it chooses between whether to treat 
# it as a vector of inputs or a single input based on the dimension of the
# first layer in the network.
function predict(neurnet, inputs::Vector{<:Number}; 
                    keepvals=false)
    if numnodesinlayer(neurnet, 1) == 1
        if !keepvals
            return predictmultiple(neurnet, inputs)
        else
            return predictmultiple_keepvals(neurnet, inputs)
        end
    else 
        if !keepvals
            return predictsingle(neurnet, inputs)
        else
            return predictsingle_keepvals(neurnet, inputs)
        end
    end
end

function predict(neurnet, inputs::Matrix; keepvals=false)
    if !keepvals
        return predictmultiple(neurnet, inputs)
    else
        return predictmultiple_keepvals(neurnet, inputs)
    end
end

function predict(neurnet, inputs::DataFrame; keepvals=false)
    return predict(neurnet, Vector.(eachrow(inputs)), keepvals)
end


In [ ]:
function predictionerror(neurnet, inputs, targets)
    numcomparisons = min(length(inputs), length(targets))
    predictions = predictmultiple(neurnet, inputs[1:numcomparisons])
    return sum(neurnet.loss.(predictions, targets[1:numcomparisons])) / numcomparisons
end

In [ ]:
"""Returns whether `inputs` and `targets` have the same length for training 
purposes. Not an exported function."""
function equaldatalengths(inputs, targets)
    return length(inputs) == length(targets)
end

"""Returns whether `inputs` and `targets` have the same length for training 
purposes. Not an exported function."""
function equaldatalengths(inputs::DataFrame, targets)
    return nrow(inputs) == length(targets)
end

In [ ]:
function updateprevweights!(neurnet::DenseNeuralNetworkSoA)
    neurnet.previousweights = deepcopy(neurnet.weights)
    neurnet.previousbias = deepcopy(neurnet.bias)
    return (neurnet.previousweights, neurnet.previousbias)
end

function forgetprevtraining!(neurnet::DenseNeuralNetworkSoA)
    neurnet.weights = deepcopy(neurnet.previousweights)
    neurnet.bias = deepcopy(neurnet.previousbias)
    neurnet.losshistory = copy(neurnet.prevlosshistory)
    return (neurnet.weights, neurnet.bias, neurnet.losshistory)
end

isfinitevec(a) = isfinite.(a)

function hasdiverged(neurnet::DenseNeuralNetworkSoA)
    return !(all(all.(isfinitevec.(neurnet.weights)) .&& all.(isfinitevec.(neurnet.bias))))
end

To train the network, we use backpropagation with stochastic gradient descent. Essentially, because a change in the weight matrix affects only the layers in front of it, we can calculate the gradient at the last layer of the network, store how we would update the weight matrix, then take that information as given so we can calculate the gradient of the next to last layer. We iterate that process backwards through the network, then used our stored information to update all of our weight matrices.

In [ ]:
function updateweights!(neurnet::DenseNeuralNetworkSoA, inputs, targets, numweightlayers,
    δatlayer, learningrate;
    preactivations=Vector{Vector{Float64}}(undef, numlayers(neurnet)),
    postactivations=Vector{Vector{Float64}}(undef, numlayers(neurnet)))

    for (input, target) in zip(inputs, targets)

        (output, _, _) = forwardpass_keepvals!(neurnet, input,
            preactivations, postactivations)

        # Following is for SoA
        outputerror = ((output .- target) .* neurnet.gradients[end](preactivations[end]))
        δatlayer[end] = outputerror

        for layerindex in (numweightlayers-1):-1:1
            δatlayer[layerindex] = ((transpose(neurnet.weights[layerindex+1])
                                     *
                                     δatlayer[layerindex+1])
                                    .*
                                    neurnet.gradients[layerindex](preactivations[layerindex]))
        end

        for (layerindex, δ, postactivation) in zip(1:numweightlayers, δatlayer, postactivations)
            # in zip(neurnet.weights, neurnet.bias, δatlayer, postactivations)
            neurnet.weights[layerindex] .-= learningrate .* δ .* transpose(postactivation)
            neurnet.bias[layerindex] .-= learningrate .* δ
        end
    end

    return neurnet
end

function train!(neurnet, inputs, targets; numepochs=10, learningrate=0.05)
    if !equaldatalengths(inputs, targets)
        error("Input and target arrays must be of the same length")
    end

    updateprevweights!(neurnet)

    numweightlayers = numlayers(neurnet) - 1
    numnodeslayer = numnodesalllayers(neurnet)

    lossatepoch = Vector{Float64}(undef, numepochs+1)
    lossatepoch[begin] = predictionerror(neurnet, inputs, targets)

    for epoch in 1:numepochs

        δatlayer = Vector{Vector{Float64}}(undef, numweightlayers)

        updateweights!(neurnet, inputs, targets, numweightlayers, δatlayer, learningrate)

        if hasdiverged(neurnet)
            forgetprevtraining!(neurnet)
            error("Model has diverged. Try turning down the learning rate.\n\
                Resetting to previous weights: $(prevweights(neurnet)) \
                | Epoch: $(epoch)")
        end

        lossatepoch[epoch+1] = predictionerror(neurnet, inputs, targets)
    end

    neurnet.prevlosshistory = neurnet.losshistory
    neurnet.losshistory = [neurnet.losshistory; lossatepoch]

    return lossatepoch
end

In [ ]:
net = DenseNeuralNetworkSoA([784, 60, 60, 10])
train!(net, trainfeatures, trainlabels; numepochs=4, learningrate=0.046)

As you can see, the error goes down sharply at first, then plateaus as the algorithm circles a local minimum of the loss function. Our final classification error is as follows:

In [ ]:
sum([(onehotdecodedigits(predict(net, feature)) == label) 
        for (feature, label) in zip(testfeatures, testdata.targets)])/length(testfeatures)